# 21 卷积层里的多输入多输出通道

通常会仔细设置的超参数
## 多输入通道：
彩色图像可能会有RGB三通道

转换为灰度可能会丢失信息

每个通道都有一个卷积核，结果是所有通道结果的和
![](./assets/2022-02-11-12-02-07.png)

![](./assets/2022-02-11-12-03-23.png)

## 多输出通道：

无论我们有多少输入通道，到目前为止我们只用到单输出通道
我们可以有多个三维卷积核，每个核生成一个输出通道
![](./assets/2022-02-11-12-05-54.png)

每个输出通道可以识别特定的模式

![](./assets/2022-02-11-12-08-37.png)

输入通道核识别并组合输入中的模式

## 1*1卷积层
![](./assets/2022-02-11-12-12-47.png)
不做空间的匹配，只在输入通道和输出通道作融合（就是一个全连接层）

## 二维卷积层
![](./assets/2022-02-11-12-15-00.png)

## 总结
输出通道数是卷积层的超参数

每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果

每个输出通道有三维卷积核 


In [2]:
# 实现一下多输入通道互相关运算

import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [3]:
# 验证互相关运算的输出

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
# 计算多个通道的输出的互相关函数

def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [6]:
# 1x1卷积

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6